# Exercise 6: SVM - Solutions

In this exercise, we see Support Vector Machine (SVM) with various kernel functions.  

We use Scikit-learn, a Python package of machine learning methods. We are using a toy binary classification example to understand Linear SVM, and then see feature expansion and kernel functions to extend it to non-linearly separable data.

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from plots import plot, plot_expand, plot_simple_data
from helpers import get_circle_dataset, get_simple_dataset
%load_ext autoreload
%autoreload 2

# 1 Scikit-Learn

Training an SVM classifer is not a easy task, so in this session, we are going to use Scikit-Learn, which is a machine learning library for Python. Most of the machine learning algorithms and tools are already implemented. In this exercise, we'll use this package to train and understand SVM. If you are interested in how to optimize a SVM, you can refer to [this](https://xavierbourretsicotte.github.io/SVM_implementation.html).

This package `sklearn` should already be implemented in your conda enviornment. If it's not the case, type the following command in your terminal:
```
conda install -y -c conda-forge sklearn
```

Scikit-Learn has modules implemented broadly for 
- Data Transformations: https://scikit-learn.org/stable/data_transforms.html
- Model Selection and Training: https://scikit-learn.org/stable/model_selection.html
- Supervised Techniques: https://scikit-learn.org/stable/supervised_learning.html
- Unsupervised Techniques: https://scikit-learn.org/stable/unsupervised_learning.html

All the magic happens under the hood, but gives you freedom to try out more complicated stuff.  
Different methods to be noted here are
- `fit`: Train a model with the data
- `predict`: Use the model to predict on test data
- `score`: Return mean accuracy on the given test data

Have a look at [this](https://scikit-learn.org/stable/tutorial/basic/tutorial.html#learning-and-predicting) for a simple example.

We will explore SVM for classification in this session: [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). We will start here with the linear kernel.

In [ ]:
# We import the SVM classifier class from scikit-learn.
from sklearn.svm import SVC

# 2 Linear SVM

SVM tries to solve linear classification problem of this form:

$$
\begin{align}
    \mathbf{w}^* = \underset{\mathbf{w},\{\xi_n\}}{\operatorname{argmin}}  \ \ & \frac{1}{2}\|\mathbf{w}\|^2 + C \sum^N_{n=1}\xi_n \\
    \operatorname{subject \  to} \ \ &  t_n\cdot(\tilde{\mathbf{w}}\cdot\mathbf{x_n}) \geq 1-\xi_n , \forall n \\
                        &\text{and  }\  \xi_n \geq 0 , \forall n
\end{align}
$$
where, $\tilde{\mathbf{w}}$ are the weights with bias term, $x_n$ is a data sample and $t_n$ is a label.

**Q.** Why do we minimize $\|\mathbf{w}\|$ ? 

**A.** $\|\mathbf{w}\|$ is inversely related to margin width.

**Q.** What is C? How should we choose the best value for C?

**A.** C is the penalty term, $\xi_i$ is an error in terms of how far data point is beyond the correct margin and $t_i \in\{-1,1\}$ the label for binary classification. We choose the right value for C, given the data, through validation set.
    
**Q.** What does it mean when $\xi_i \gt 0$ ?

**A.** Our data may not be linearly separable, hence maximizing the margin will lead to some misclassifications. $\xi_i$ is greater than zero when a data point is within the margin, or misclassified, and how many such data points are allowed is controlled by C. 


## 2.1 Binary Classification

Let's begin with a simple **binary** classification using Linear SVM.
The data is simply **linearly** separable.

We visualize here the optimal maximum-margin solution without misclassifications.

In [ ]:
# Simple data with 3 points per class
X = np.array([[2,4],[1,4],[2,3],[6,-1],[7,-1],[5,-3]] )
Y = np.array([-1, -1, -1, 1, 1, 1])
plot_simple_data()

In this part, you are asked to build a SVM classifier using SVC and to understand the outputs from the fitted model.

In [ ]:
### WRITE YOUR CODE HERE
# 1. Declare a SVC with C=1.0 and kernel='linear'
clf = SVC(C=1.0, kernel='linear')

# 2. use x and y to fit the model
clf.fit(X, Y) 

# 3. show the fitted model
plot(X, Y, clf)

# Some information we can extract from the model
# Take note of them as you might need them in the future!
print('w = ', clf.coef_)
print('w0 = ', clf.intercept_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Support vectors = ', clf.support_vectors_)
print('Indices of support vectors = ', clf.support_)

In this case, we found that we have 2 **support vectors**, one in each class. They are shown highlighed in green in the plot. A support vector is a data sample that is either on the margin or within the margin (or misclassified). 

Let's inspect the result of the classification. We do the classification in the following way:

$$ 
y_i = \begin{cases}
-1 & \text{if} \ \mathbf{x}_i^T \mathbf{w} + w_0 < 0\\
1 & \text{otherwise}
\end{cases}
$$

*Note*: when doing this on multiple data points at a time, $X$ is an $N\times D$ matrix.

In [ ]:
# Use the weights (w) from the fitted model to predict the labels of input data points
def raw_predict(X, w, w0):
    '''
    Given input data X, SVM weight w and w0, output the prediction result.
    
    Args:
        X: data, array of shape (N, D) where N is the number of datapoints and D is the dimension of features.
        w: weights, array of shape (D,)
        w0: bias, array of shape (1,)
    Returns:
        out: predictions, array of shape (N,)
    '''
    ### WRITE YOUR CODE HERE
    out = np.sign(np.dot(X, w) + w0)
    return out.astype(int)

x_test = np.array([
    [4, 2],
    [ 6, -3]
])

### WRITE YOUR CODE HERE: Use your implementation to do the prediction on the test data.
raw_pred = raw_predict(x_test, clf.coef_[0], clf.intercept_)
print("Prediction from your implementation: ", raw_pred)

### WRITE YOUR CODE HERE: Use scikit-learn's predict function to do the prediction on the test data.
model_predict = clf.predict(x_test)

print("Prediction from the model: ", model_predict)

assert np.isclose(raw_pred, model_predict).all(), "Your implementation is not correct."


Now, let us determine the indices of the support vectors. (Reminder: These are the data samples that fall on the margin or within the margin). 

In [ ]:
## We can also calculate the decision function manually.

## Step 1
### WRITE YOUR CODE HERE: Code the decision function: Xw + w_0
decision_function = np.dot(X, clf.coef_[0]) + clf.intercept_

## Step 2: We can also retrieve the decision function from the model:
decision_function_from_model = clf.decision_function(X)

assert np.isclose(decision_function, decision_function_from_model).all(), "Your implementation is not correct."

# What condition do the support vectors satisfy? 
# Remember that the support vectors are the points that on to the decision boundary, or within the margin.
### WRITE YOUR CODE HERE, hint: look into np.nonzero
support_vector_indices = np.nonzero(Y * decision_function <= 1.)

print('I find the indices of support vectors = ', support_vector_indices)
assert np.isclose(support_vector_indices, clf.support_).all(), "Your implementation is not correct."

## 2.2 Different C values

Let's explore the effect of $C$ on a different dataset.

**Q.** How do you expect the margin to vary with C? *Hint*: have a look at the optimization formulation above.

**A.** Lower C allows more misclassifications and hence lead to larger margins, while bigger C reduces misclassfications and hence lead to smaller margins.

In [ ]:
# Get the simple dataset
X, Y = get_simple_dataset()
plot(X, Y, None, dataOnly=True)

In the code below, vary the C value from 0.001 to 10 and pay attention to the changes.

The plot shows the decision boundary and margins of the learnt model. Encircled points are the support vectors.  
WARNING: if the margins go beyond the limits of the axis, they might not be shown.

In [ ]:
# Declare a SVM model with linear kernel and PLAY WITH THE VALUE OF C
clf = SVC(kernel='linear', C=0.01)

# Call the fit method
clf.fit(X, Y)

# Plot the decision boundary
plot(X, Y, clf)


### Additional Reading (if interested)
- Multiclass SVM (Bishop- Multiclass SVMs 7.1.3)
- Can we have probabilistic interpretation of SVM? (Bishop- Relevance Support Machine 7.2)

# 3 Kernel SVM

Beyond the linear problem we discussed before, SVM can also solve non-linear classification problem by doing some feature expansion on the input data. 

We replace $\mathbf{x}_i$ with $\phi(\mathbf{x}_i)$, and then $\mathbf{x}_i^\top\mathbf{x}_j$ with $\phi(\mathbf{x}_i)^\top\phi(\mathbf{x}_j)=k(\mathbf{x}_i,\mathbf{x}_j)$. 

$\phi(\cdot)$ is the (possibly unknown) feature expansion function, and $k(\cdot)$ is the kernel function.

The **dual form** of this problem is given by:

$$
\begin{align}
    \underset{\{\lambda_i\}}{\operatorname{max}} \ \ 
    & \sum_{n=1}^N \lambda_i - \frac 1 2 \sum_{i=1}^N\sum_{j=1}^N \lambda_i\lambda_jt_it_jk(\mathbf{x}_i,\mathbf{x}_j)  \\   
    \operatorname{subject \ to} & \ \ \sum_{i=1}^N \lambda_it_i = 0 \\
                 & \ \ 0 \leq \lambda_i \leq C, \forall i \ \ 
\end{align}
$$

**Q.** 
1. How can you write $\mathbf{w}$ using $\lambda_i$ and function $\phi$?
2. How is $y(\mathbf{x})$ represented using $\lambda_i$?
 
**A.**
1. $\mathbf{w} = \sum_{i=1}^N \lambda_it_i\phi(\mathbf{x}_i) $
2. We plugging the $\mathbf{w}$ as, 
  $$
  \begin{align}
    y(\mathbf{x}) &= \mathbf{w}^\top\phi(\mathbf{x}) + w_0 \\
                        &= \sum_{i=1}^N \lambda_it_ik(\mathbf{x},\mathbf{x_i}) + w_0
  \end{align}
  $$
   * The sum can be computed on the support vectors ($\delta$) only:
     $$
     \begin{align}
       y(\mathbf{x}) & = \sum_{i \in \delta} \lambda_it_ik(\mathbf{x},\mathbf{x_i}) + w_0
     \end{align}
     $$

We continue with the Scikit-Learn implementation of [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). The main parameters you should look for are:
- Kernel Functions: `kernel`. Linear, Polynomial and RBF ($X$ is the data)
    - Linear: `linear`. $\langle X, X' \rangle $.
    - Polynomial: `poly`. $( \gamma \langle X, X' \rangle + r)^d $. $d$ is specified by keyword `degree`, $r$ by `coef0`.
    - RBF: `rbf`. $\exp(-\gamma ||X - X'||^2)$. $\gamma$ is specified by keyword `gamma`, must be greater than 0.
- Penalty term, `C`: for all
- `gamma`: for Polynomial and RBF kernel (mostly RBF)
- `degree`: for Polynomial kernel


## 3.1 Non-linearly separable data

We use a binary dataset that cannot be separated linearly in the original feature space.

Then, we'll try the different kernel and explore how their parameters affect the results.

In [ ]:
# Load data
X, Y = get_circle_dataset()
plot(X, Y, None, dataOnly=True)

## 3.2 Linear SVM

As you should expect, linear SVM does not perform well in this case.

In [ ]:
# Use SVM with linear kernel
clf_linear = SVC(kernel='linear', C=1.0)
    
clf_linear.fit(X, Y)
plot(X, Y, clf_linear)

## 3.3 Polynomial SVM

For polynomial SVM, we have two options:
1. We can explicitely write a polynomial feature expansion $\phi_\text{poly}(\cdot)$ to edit the data $X$, then use linear SVM on it.
2. We use the kernel trick to only define a kernel function $k_\text{poly}(\cdot,\cdot)$, which is directly used in SVM.

Let's do both and compare the results!

Fill in the function `expand_X()` that performs polynomial feature expansion. 
You should add a bias term, but **omit the interaction terms**. An example:

For $D=2$ and $\text{degree}=3$, the data
$$
\mathbf{x}_i = \begin{bmatrix}\mathbf{x}_i^{(0)}& \mathbf{x}_i^{(1)}\end{bmatrix},
$$
after the polynomial feature expansion, will become
$$ 
\mathbf{\phi}(\mathbf{x}_i) = \begin{bmatrix}\mathbf{1} & \mathbf{x}_i^{(0)} & \mathbf{x}_i^{(1)} & (\mathbf{x}_i^{(0)})^2 & (\mathbf{x}_i^{(1)})^2 & (\mathbf{x}_i^{(0)})^3 & (\mathbf{x}_i^{(1)})^3 \end{bmatrix}.
$$

In [ ]:
# Perform degree-d polynomial feature expansion of input data X
def expand_X(X, degree):
    """
    Polynomial feature expansion with bias but omitting interaction terms
    
    Args:
        X (array): data, shape (N, D).
        degree (int): The degree of the polynomial feature expansion.
    Returns:
        expanded_X (array): Expanded data with shape (N, new_D), 
                               where new_D = D * degree + 1
    """
    ### WRITE YOUR CODE HERE
    expanded_X = np.concatenate([X**i for i in range(1, degree+1)], axis=1)
    expanded_X = np.concatenate([np.ones((X.shape[0], 1)), expanded_X], axis=1)
    return expanded_X

In [ ]:
# Polynomial SVM
degree = 2 # you can play with the degree. How does the decision boundary change?

## Do polynomial feature expansion
### WRITE YOUR CODE HERE
expanded_X = expand_X(X, degree)

print("The original data has {} features.".format(X.shape[1]))
print("After degree-{} polynomial feature expansion (with bias, without interaction terms) the data has {} features.".format(degree,expanded_X.shape[1]))

## Use SVM with linear kernel on expanded data
### WRITE YOUR CODE HERE: you can play with C
expanded_clf = SVC(kernel='linear', C=1.0)
expanded_clf.fit(expanded_X, Y)

plot_expand(X, Y, expanded_clf, degree=degree)

The non-linearly separable dataset can now be classified correctly by a linear SVM, thanks to the polynomial feature expansion.

Let's now directly use the polynomial kernel function in SVM.

Given data $\mathbf{X}$ with $N$ samples, its kernel matrix $\mathbf{K}$ is the $N \times N$ symmetric Gram matrix with elelments 

$$ \mathbf{K}_{n,m} = \phi(\mathbf{x}_n)^T\phi(\mathbf{x}_m) = k(\mathbf{x}_n, \mathbf{x}_m) $$

The polynomial kernel is SVM is written as:
- poly: $( \gamma \langle \mathbf{X}, \mathbf{X'} \rangle + r)^d $. $d$ is specified by keyword `degree`, $r$ by `coef0`,
   
where $X$ is the data.

Note that $\phi$ **does not appear explicitly** in the kernel functions!

In [ ]:
# Use SVM with poly kernel
### WRITE YOUR CODE HERE and PLAY with C, degree and coef0 parameters
clf_poly = SVC(kernel='poly', C=1.0, degree=2, coef0=1.0)
    
clf_poly.fit(X, Y)
plot(X, Y, clf_poly)

**Q.** 
1. What are the differences between polynomial feature expansion and polynomial kernel function? 

2. Is the SVM trained with linear kernel on polynomially expanded data same as the SVM trained with polynomial kernel function on original data?

**A.** 
1. Polynomial feature expansion explicitly computes the new features $\phi(\mathbf{x}_i)$ and performs linear SVM in the new feature space. Polynomial kernel function uses a function $k(\mathbf{x}_i, \mathbf{x}_j) = \phi(\mathbf{x}_i)^\top\phi(\mathbf{x}_j)$ to compute a scalar product between two data points in the feature space, without having to explicitly send them there.

2. Yes, the 2 SVMs are similar. There might be small differences based on how the polynomial kernel function is written, or how the polynomial feature expansion is done, but the results should be equivalent.

## 3.4 RBF SVM

Finally, let's try the Radial Basis Function (RBF) kernel:
* `rbf`: $\exp(-\gamma ||X - X'||^2)$. $\gamma$ is specified by keyword `gamma`, and must be greater than 0.

$\gamma$ is a form of scaling factor for the distance between points. If it is increased, then the exponential decays faster with distance, and vice-versa. 

Try different values of $\gamma$ below, e.g., in the range $[0.01, 100]$, and see how the decision function is affected.

In [ ]:
# Use SVM with rbf kernel
### WRITE YOUR CODE HERE and PLAY with C and gamma parameters
clf_rbf = SVC(kernel='rbf', C=1.0, gamma=1.0)
    
clf_rbf.fit(X, Y)
plot(X, Y, clf_rbf)

### Choosing a kernel

We have seen how to use SVM for classification as a linear classifier, and then how to extend it to non-linearly separable data through the use of kernel functions.

But in practice, **how does one choose which kernel to use and its hyperparameters?**

You guessed it $\rightarrow$ by using a validation set! 

While in this exercise we have only tested SVM on the training data to visualize the effect of the different hyperparameters and kernels, on real problems we would also use validation data to evaluate the performance of the classifier.

# 4 Written questions

## 4.1 Find the Margins

We learn a hard-margin (i.e., *no misclassification allowed*) linear SVM classifier for the data samples shown in the figure below.

<img src="img/svm_q1.png" width="400">

Answer the following questions:

1. What do you expect the decision boundary to be? What is its equation in terms of $x_1$ and $x_2$?

2. How many support vectors are there? Write down their coordinates.

3. What are the weights (including the bias term) for the SVM formulation? *Hint*: use the decision boundary found above and that the support vectors should lie exactly on the margin.

4. If we learn a **soft-margin** (with slack variables) linear SVM classifier, then what can be the maximum and the minimum number of support vectors, assuming we play with the penatly term $C$?

**Answers.**

1. The decision boundary has to be perpendicular to the line joining the closest points and also divide it equally. Hence we have $x_1-x_2=0$, passing through (3,3) and the origin.

2. There are two support vectors: the points (4,2) and (2,4).

3. Let $w_1$, $w_2$ be the two weights and $b$ the bias. We know that (3,3) and (0,0) lie on decision boundary, so they satisfy $\mathbf{w}^T\mathbf{x} + w_0 = 0$. That is, 
$$
\begin{align} 
3w_1+3w_2+w_0 &= 0 \\ 
0w_1+0w_2+w_0 &= 0 
\end{align} 
$$
which gives $w_1=-w_2$ and $w_0=0$. 

Now, since (4,2) and (2,4) are support vectors, we have
$$ 
\begin{align} 
4w_1+2w_2+w_0 &= 1 \\ 
2w_1+4w_2+w_0 &= -1 
\end{align}
$$
which gives $w_1=\frac{1}{2}$ and $w_2=-\frac{1}{2}$.

4. We have a maximum of 4, as all data points can be support vectors when C is very small, and a minimum of 2 (one on each side of the decision boundary).
 


**Q.2** (MCQ) Recall that the SVM formulation is written as

$$
\begin{align}
    \underset{\mathbf{w},\{\xi_n\}}{\operatorname{min}}  \ \ & \frac{1}{2}\|\mathbf{w}\|^2 + C \sum^N_{n=1}\xi_n \\
    \operatorname{subject \  to} \ \ &  t_n\cdot(\mathbf{w}\cdot\mathbf{x_n} + w^{(0)}) \geq 1-\xi_n , \forall n \\
                        &\text{and  }\  \xi_n \geq 0 , \forall n
\end{align}
$$

where $C$ is a constant, $\xi_{n}$ are the slack variables, $\mathbf{x}_n$ is a data sample, $t_n$ is a target label in $\{-1,1\}$, $w^{(0)}$ is the bias term, and $\mathbf{w}$ are the parameters of the model (without the bias). 

Which of the following statements regarding this formulation is/are correct?
1. Minimizing $\frac{1}{2}\|\mathbf{w}\|^2$ allows us to *minimize* the margin between the decision boundary and the data points.
2. There are as many slack variables $\xi_n$ as there are support vectors.
3. While training the SVM, we are optimizing for the bias $w^{(0)}$, the model parameters $\mathbf{w}$, and the slack variables $\xi_n$.
4. If $C$ is $0$, the points cannot be misclassified.
5. The minimization of $C \sum_{n=1}^N \xi_{n}$ aims to *minimize* the number of points that are misclassified.

**A.2**
1. *Wrong*: it's to *maximize* the margin.
2. *Wrong*: there are as many as data points. However, there are as many *non-zero* $\xi_n$ as support vectors.
3. *Correct*: the optimization is over all of these parameters.
4. *Wrong*: if $C=0$, then we do not penalize at all the misclassifications or points inside the margin. Therefore, there might be a lot of misclassifications.
5. *Correct*: the $\xi_n$ are slack variables that allow misclassifications or points inside the margin. By aiming to minimize them, we are trying to reduce misclassification.